In [1]:
import os

setting_proxy=True

if setting_proxy:
    proxy_address = 'http://127.0.0.1:7890'
    os.environ['http_proxy'] = proxy_address
    os.environ['https_proxy'] = proxy_address

In [2]:
#from news_api import search_news, search_headlines

from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm-6b", trust_remote_code=True)
chat_model = AutoModel.from_pretrained("THUDM/chatglm-6b",trust_remote_code=True ).half().cuda()

from pprint import pprint
from paddlenlp import Taskflow
schema = ['name', 'time', 'location','organization',"product","date"] # Define the schema for entity extraction
ie = Taskflow('information_extraction', schema=schema, model="uie-base-en")

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
sim_model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')

/usr/local/miniconda3/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Loading checkpoint shards: 100%|██████████| 8/8 [00:07<00:00,  1.07it/s]
/usr/local/miniconda3/lib/python3.8/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
[2023-06-19 10:40:

In [3]:
import requests as r

import os

setting_proxy=True
news_key='097e4195453c47bb8c3ef0e6427493a7'

if setting_proxy:
    proxy_address = 'http://127.0.0.1:7890'
    os.environ['http_proxy'] = proxy_address
    os.environ['https_proxy'] = proxy_address


def search_news(keywords,from_date='2023-05-20',to_date='2023-06-18'):
    url = 'https://newsapi.org/v2/everything'
    params = {
        'q': keywords,
        'from': from_date,
        'to': to_date,
        'sortBy': 'popularity',#   'publishedAt', # popularity
        'apiKey': news_key,
        'language':'en'
    }
    try:
        response = r.get(url, params=params)
        # Process the response as needed
        return response.json()["articles"]
    except r.exceptions.RequestException as e:
        print('An error occurred:', e)

def search_headlines(from_date='2023-05-20',to_date='2023-06-18'):
    url = 'https://newsapi.org/v2/top-headlines'
    params = {
        #'q': keywords,
        'from': from_date,
        #'to': '2023-06-18',
        #'sortBy': 'popularity',#   'publishedAt', # popularity
        'apiKey': news_key,
        'country': 'us',
        'category': "business",
        'language':'en'
    }
    
    try:
        response = r.get(url, params=params)
        # Process the response as needed
        return response.json()["articles"]
    except r.exceptions.RequestException as e:
        print('An error occurred:', e)

In [4]:
def chatbot(query="",history=[]):
    response=chat_model.chat(tokenizer,query,history)
    #print(response[0])
    return response[0]

def sentence_rank(sentences):
    sentence_embeddings = sim_model.encode(sentences)
    similarity_matrix = cosine_similarity(sentence_embeddings)
    first_sentence_similarity_scores = similarity_matrix[0]
    sorted_indexes = first_sentence_similarity_scores.argsort()[::-1]
    sorted_sentences = [sentences[i] for i in sorted_indexes]
    return sorted_sentences

def IE(query):
    entities=""
    category=""
    time_period=""
    ans=ie(query)
    for kk in ans[0].keys():
        if kk=="date":
            time_period=ans[0]["date"][0]["text"]
            continue
        for ss in ans[0][kk]:
            entities+=(ss["text"]+" ")
    
    topics=["business","entertainment","general health","science","sports","technology"]
    for ss in topics:
        if ss in query:
            category=ss
            break
    if entities!=0:
        print("Searching: ",entities, category, time_period)
    else:
        print("Searching: ","Headlines ", category, time_period)
    return entities, category, time_period

from datetime import datetime
def date_understanding(date_string):
    date=chatbot(date_string,history=[("Today is June 19th, 2023. Please convert my date description into a standard date string. Please only answer a standard date string","Sure! The standard date string for June 19th, 2023 is '2023-06-19'."),("July 5th","2023-07-05"),("September 10th, 2022","2022-09-10"),("first week in November","2023-11-07"),("last month","2023-05-19"),("today","2023-06-19")])
    try:
        
        start_date = datetime.strptime('2023-05-20', '%Y-%m-%d')
        end_date = datetime.strptime('2023-06-18', '%Y-%m-%d')
        if start_date <= datetime.strptime(date, '%Y-%m-%d') <= end_date:
            return True, date
        elif start_date >= datetime.strptime(date, '%Y-%m-%d'):
            return True, "2023-05-20"
        else:
            return True, "2023-06-18"
        return True, date
    except ValueError:
        return False, ""

IE("What recent news is there about business yesterday?")

Searching:   business yesterday


('', 'business', 'yesterday')

In [5]:
def chat_news(query):
    entities, category, time_period=IE(query)
    valid_date=False
    if len(time_period)!=0:
        valid_date,date=date_understanding(time_period)
    if not valid_date:
        date="2023-05-20"
    if len(entities)!=0:
        re=search_news(entities,date)
    elif len(category)!=0:
        re=search_news(entities+category,date)
    else:
        re=search_headlines(date)
    sentences=[query]
    for art in re:
        sentences.append(art["description"])
    retrieved_sents=sentence_rank(sentences)
    prompt="Here are related news:\n"
    for i in range(5):
        prompt+="\n#"+str(i)+"\n"
        prompt+=retrieved_sents[i+1]
    prompt+="\n\nAccording to the above news, tell me "
    prompt+=query
    prompt+="\n"
    #print(prompt)
    response=chatbot(prompt)
    print(response)
    #return response[0]

In [6]:
chat_news("What are the top news stories yesterday?")

The dtype of attention mask (torch.int64) is not bool


Searching:    yesterday
It appears that the topics of interest for yesterday include:

1. Microsoft outages and cyberattack
2. Reddit blackout
3. Elon Musk's opinions
4. BlackRock applying for a spot Bitcoin ETF
5. U.S. representative introducing bill to fire SEC's chief Gary Gensler

These topics may have been covered in the news articles you provided. However, it is important to note that the accuracy and completeness of the information in the news articles may vary.


In [7]:
chat_news("Is it true that Donald Trump has recently been accused of a crime after June 5th?")

Searching:  Trump   June 5th
Yes, it is true that Donald Trump has been indicted on seven counts in a special counsel probe into his handling of a cache of classified documents. The indictment was announced on June 10, 2021. However, it is important to note that this is a separate case from the ongoing investigations into Russian interference in the 2016 election, and whether Trump's campaign collaborated with the Russian government. It is also possible that the charges in this case are related to other matters that have not been publicly disclosed.


In [8]:
chat_news("What recent news is there about Elon Musk?")

Searching:  Musk   
There are several recent news articles about Elon Musk. Here are some of the main events:

* A shareholder lawsuit against Elon Musk has been thrown out by a judge.
* Elon Musk was scheduled to visit Tesla's Shanghai plant on Wednesday night.
* He launched his bid for the White House on Twitter, but it has been revealed that the platform has technical and political limitations.
* Elon Musk hosts an event for an anti-vaxx Democratic candidate on Twitter.

Overall, it appears that Elon Musk is involved in several different projects and events in the past few weeks.


In [9]:
chat_news("Which company recently released the Vision Pro product?")

Searching:  Vision Pro   
Apple released the Vision Pro product.


In [11]:
chat_news("What business news has there been in the past 7 days?")

Searching:   business the past 7 days
There have been several business news articles in the past 7 days related to:

* The SEC launching two lawsuits over retail crypto trading
* A change to Facebook's algorithm causing a drop in traffic to news and media websites
* The start of Summer Game Fest and its impact on the gaming industry
* First Citizens announced job cuts after buying the US operations of another troubled lender
* Microsoft's OpenAI deal, with GPT-4-backed chatbot and generative capabilities coming to Office products

Please note that this is only a partial list and there may be other business news articles that I have not covered.
